# SCRAPING WEB

#### In this section, we will to scrap the table: List of postal codes of Canada from [*wikipedia*.]('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [1]:
#IMPORTING LIBRARIES
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
import numpy as np


In [2]:
#SENDING A HTTP REQUEST TO A URL
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r= requests.get(url).text

#PARSE THE HTML CONTENT
soup= BeautifulSoup(r, "html.parser")

can_table= soup.find('table', class_= 'wikitable sortable')
columnas=[]
filas=[]
#SCRIPTING TABLE FROM WEB
for tr in can_table.find_all('tbody'):
        for th in tr.find_all('th'):
            columnas.append([th.text.strip()])
        
        for row in tr.find_all('td'):
            filas.append([row.text.strip()])  
       
def split(arr, size):
     arrs = []
     while len(arr) > size:
         pice = arr[:size]
         arrs.append(pice)
         arr   = arr[size:]
     arrs.append(arr)
     return arrs
filas= split(filas, 3)        
pc=[]
br=[]
nb=[]
for i in range(len(filas)):
    pc.append(filas[i][0][0])
    br.append(filas[i][1][0])
    nb.append(filas[i][2][0])

#CREATE DATAFRAME
df=pd.DataFrame(columns=['Postal Code', 'Borough','Neighborhood'])
df['Postal Code']=pc
df['Borough']=br
df['Neighborhood']=nb

df.replace("Not assigned", np.nan, inplace = True)
df.dropna(subset=["Borough"], axis=0, inplace=True)    #Drop rows in Borough where there are null values
df.reset_index(inplace=True) 
df.drop('index', axis=1, inplace=True)


# 1. __THIS IS THE FINAL TABLE TO EVALUATE__

In [3]:
df.head(12)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


#### Now, let's see if we have more rows with null values in all columns

In [4]:
df.isnull().sum()

Postal Code     0
Borough         0
Neighborhood    0
dtype: int64

#### Let's see our dataframe shape

In [5]:
df.shape

(103, 3)

# FOURSQUARED LOCATION DATA 

### In this section, we will get the latitude and the longitude coordinates of each neighborhood. 

In [6]:
import json # library to handle JSON files
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm  
import matplotlib.colors as colors
#import kmeans from clustering stage
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: | 
  - anaconda/osx-64::ca-certificates-2020.1.1-0, anaconda/osx-64::certifi-2019.11.28-py37_0, anaconda/osx-64::openssl-1.1.1d-h1de35cc_4
  - anaconda/osx-64::ca-certificates-2020.1.1-0, anaconda/osx-64::openssl-1.1.1d-h1de35cc_4, defaults/osx-64::certifi-2019.11.28-py37_0
  - anaconda/osx-64::certifi-2019.11.28-py37_0, anaconda/osx-64::openssl-1.1.1d-h1de35cc_4, defaults/osx-64::ca-certificates-2020.1.1-0
  - anaconda/osx-64::openssl-1.1.1d-h1de35cc_4, defaults/osx-64::ca-certificates-2020.1.1-0, defaults/osx-64::certifi-2019.11.28-py37_0
  - anaconda/osx-64::certifi-2019.11.28-py37_0, defaults/osx-64::ca-certificates-2020.1.1-0, defaults/osx-64::openssl-1.1.1d-h1de35cc_4
  - defaults/osx-64::ca-certificates-2020.1.1-0, defaults/osx-64::certifi-2019.11.28-py37_0, defaults/osx-64::openssl-1.1.1d-h1de35cc_4
  - anaconda/osx-64::ca-certificates-2020.1.1-0, anaconda/osx-64::certifi-2019.11.28-py37_0, defaults/osx-64::openssl-1.1.1d-h1de35cc_4
  - anaconda/osx-64::ca-

### To get the latitude and the longitude coordinates of each postal code, we will download a csv file that has the geographical coordinates of each postal code. 

In [7]:
df_gp=pd.read_csv('http://cocl.us/Geospatial_data')
df_gp.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Next, we will add the latitude and the longitude coordinates to our initial Dataframe

# 2. __THIS IS THE FINAL TABLE TO EVALUATE__

In [8]:
df_final= pd.merge(df, df_gp, on='Postal Code')
df_final.head(12)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


# 3. Explore and cluster the neighborhoods in Toronto

### In this last section, we will explore the neighborhoods in Toronto using Foursquare API. Then,we will group the borough into clusters using the k-means clustering algorithm 

In [9]:
address = 'Toronto, ONT'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of TORONTO are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of TORONTO are 43.678523999999996, -79.62912913064454.


#### Now, let's create a Toronto Maps using Folium library

In [10]:
# create map of TORONTO using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

In [11]:
# add markers to map
for lat, lng, borough, neighborhood in zip(df_final['Latitude'], df_final['Longitude'], df_final['Borough'], df_final['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### To access to Foursquare API, we need to know our credentials 

In [12]:
CLIENT_ID = 'PM0RLBSUM2LF3CTMUYTM1MUYUK0HP1X1MJVIBRZWTR2O3EE1' # your Foursquare ID
CLIENT_SECRET = 'TV20HSNRUBWNDKRSEJH5Q4E5DUP4UJC5MBFQKCVPZPDXTG4Z' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PM0RLBSUM2LF3CTMUYTM1MUYUK0HP1X1MJVIBRZWTR2O3EE1
CLIENT_SECRET:TV20HSNRUBWNDKRSEJH5Q4E5DUP4UJC5MBFQKCVPZPDXTG4Z


In [13]:
#See how many Borough there are  
df_final["Borough"].unique()

array(['North York', 'Downtown Toronto', 'Etobicoke', 'Scarborough',
       'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

In [14]:
df_final.groupby("Borough").count()

,Postal Code,Neighborhood,Latitude,Longitude
Borough,,,,
Central Toronto,9,9,9,9
Downtown Toronto,19,19,19,19
East Toronto,5,5,5,5
East York,5,5,5,5
Etobicoke,12,12,12,12
Mississauga,1,1,1,1
North York,24,24,24,24
Scarborough,17,17,17,17
West Toronto,6,6,6,6



Realize there is only one row for 'Mississauga' in our dataset, so let's delete that row.



In [15]:
df_final.set_index('Borough', inplace=True)
df_final.drop('Mississauga', axis=0, inplace=True) #Drop Mississauga
df_final.reset_index(level=0, inplace=True)

In [16]:
df_final.groupby("Borough").count() #Our new df without Mississauga

,Postal Code,Neighborhood,Latitude,Longitude
Borough,,,,
Central Toronto,9,9,9,9
Downtown Toronto,19,19,19,19
East Toronto,5,5,5,5
East York,5,5,5,5
Etobicoke,12,12,12,12
North York,24,24,24,24
Scarborough,17,17,17,17
West Toronto,6,6,6,6
York,5,5,5,5


In [17]:
df_final.head(12)

,Borough,Postal Code,Neighborhood,Latitude,Longitude
0,North York,M3A,Parkwoods,43.753259,-79.329656
1,North York,M4A,Victoria Village,43.725882,-79.315572
2,Downtown Toronto,M5A,"Regent Park, Harbourfront",43.654260,-79.360636
3,North York,M6A,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,Downtown Toronto,M7A,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,Etobicoke,M9A,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,Scarborough,M1B,"Malvern, Rouge",43.806686,-79.194353
7,North York,M3B,Don Mills,43.745906,-79.352188
8,East York,M4B,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,Downtown Toronto,M5B,"Garden District, Ryerson",43.657162,-79.378937


#### Let's work with Downtown Toronto, West Toronto, East Toronto and Central Toronto and call it just Toronto


In [18]:
df_final.set_index(['Borough'], inplace=True)
toronto_data = df_final.loc[['Central Toronto', 'Downtown Toronto', 'East Toronto', 'West Toronto']]

In [19]:
toronto_data.reset_index(level=0, inplace=True)

In [20]:
toronto_data['Borough']= 'Toronto'

In [21]:
toronto_data.drop('Postal Code', axis=1, inplace=True)

I decided to explore only this region: 

In [22]:
toronto_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Toronto,Lawrence Park,43.728020,-79.388790
1,Toronto,Roselawn,43.711695,-79.416936
2,Toronto,Davisville North,43.712751,-79.390197
3,Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307
4,Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678


In [23]:
# create map of TORONTO grouping by Borough
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

In [24]:
# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Let's borrow the get_category_type function from the Foursquare lab.

In [25]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Let's borrow the getNearbyVenues function from the Foursquare lab. to get all the neighborhoods in Toronto.

In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [27]:
LIMIT=100
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley
The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Business reply mail Processing Centre, South Central Letter

In [28]:
print(toronto_venues.shape)
toronto_venues.head()

(1622, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School
2,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
3,Roselawn,43.711695,-79.416936,Rosalind's Garden Oasis,43.712189,-79.411978,Garden
4,Roselawn,43.711695,-79.416936,Havergal College,43.712108,-79.411680,Music Venue


Let's check how many venues were returned for each neighborhood

In [29]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
"Brockton, Parkdale Village, Exhibition Place",24,24,24,24,24,24
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",17,17,17,17,17,17
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,65,65,65,65,65,65
Christie,16,16,16,16,16,16
Church and Wellesley,77,77,77,77,77,77
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,35,35,35,35,35,35


Let's check how many venues were returned for each neighborhood

In [30]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 233 uniques categories.


### Analyze Each Neighborhood

In [31]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]


toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.015385,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.015385,0.000000,0.000000,0.015385,0.000000,0.000000
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.025974,0.012987,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012987,...,0.012987,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.012987,0.000000
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,...,0.000000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.028571,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [33]:
#Funtion to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [34]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Café,Cheese Shop,Bakery,Seafood Restaurant,Restaurant,Beer Bar,Lounge,Park
1,"Brockton, Parkdale Village, Exhibition Place",Café,Bakery,Breakfast Spot,Coffee Shop,Grocery Store,Stadium,Burrito Place,Restaurant,Climbing Gym,Pet Store
2,"Business reply mail Processing Centre, South C...",Yoga Studio,Auto Workshop,Skate Park,Light Rail Station,Burrito Place,Farmers Market,Fast Food Restaurant,Butcher,Restaurant,Recording Studio
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Sculpture Garden,Boutique,Rental Car Location,Plane,Coffee Shop,Boat or Ferry,Harbor / Marina
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Japanese Restaurant,Café,Bar,Burger Joint,Salad Place,Thai Restaurant,Department Store


## Cluster Neighborhoods

Run k-means to cluster the neighborhood into 4 clusters.

In [35]:
# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

In [36]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Toronto,Lawrence Park,43.728020,-79.388790,0,Park,Swim School,Bus Line,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run
1,Toronto,Roselawn,43.711695,-79.416936,1,Garden,Music Venue,Women's Store,Dance Studio,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
2,Toronto,Davisville North,43.712751,-79.390197,2,Park,Gym,Breakfast Spot,Dog Run,Sandwich Place,Food & Drink Shop,Hotel,Department Store,Deli / Bodega,Eastern European Restaurant
3,Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307,0,Park,Jewelry Store,Trail,Sushi Restaurant,Dance Studio,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run
4,Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678,2,Clothing Store,Coffee Shop,Grocery Store,Sporting Goods Shop,Café,Chinese Restaurant,Diner,Fast Food Restaurant,Mexican Restaurant,Park


Finally, let's visualize the resulting clusters


In [37]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

###  Examine Clusters


## CLUSTER 1

In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(4, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Lawrence Park,0,Park,Swim School,Bus Line,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run
3,"Forest Hill North & West, Forest Hill Road Park",0,Park,Jewelry Store,Trail,Sushi Restaurant,Dance Studio,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run
7,"Moore Park, Summerhill East",0,Gym,Trail,Women's Store,Dance Studio,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run
23,Rosedale,0,Park,Playground,Trail,Cupcake Shop,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
28,The Beaches,0,Health Food Store,Trail,Pub,Women's Store,Dance Studio,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run


### CLUSTER 2

In [44]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(4, toronto_merged.shape[1]))]]



,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Roselawn,1,Garden,Music Venue,Women's Store,Dance Studio,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


### CLUSTER 3

In [45]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(4, toronto_merged.shape[1]))]]



,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Davisville North,2,Park,Gym,Breakfast Spot,Dog Run,Sandwich Place,Food & Drink Shop,Hotel,Department Store,Deli / Bodega,Eastern European Restaurant
4,"North Toronto West, Lawrence Park",2,Clothing Store,Coffee Shop,Grocery Store,Sporting Goods Shop,Café,Chinese Restaurant,Diner,Fast Food Restaurant,Mexican Restaurant,Park
5,"The Annex, North Midtown, Yorkville",2,Coffee Shop,Café,Sandwich Place,BBQ Joint,History Museum,Flower Shop,Indian Restaurant,Donut Shop,Liquor Store,Middle Eastern Restaurant
6,Davisville,2,Pizza Place,Dessert Shop,Sandwich Place,Sushi Restaurant,Coffee Shop,Gym,Italian Restaurant,Café,Seafood Restaurant,Brewery
8,"Summerhill West, Rathnelly, South Hill, Forest...",2,Pub,Coffee Shop,Fried Chicken Joint,Liquor Store,Restaurant,Sports Bar,Bank,Bagel Shop,Supermarket,Sushi Restaurant
9,"Regent Park, Harbourfront",2,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Café,Theater,Yoga Studio,Cosmetics Shop,Shoe Store
10,"Queen's Park, Ontario Provincial Government",2,Coffee Shop,Sushi Restaurant,Diner,Arts & Crafts Store,Burrito Place,Distribution Center,Discount Store,Smoothie Shop,Café,Beer Bar
11,"Garden District, Ryerson",2,Clothing Store,Coffee Shop,Cosmetics Shop,Café,Middle Eastern Restaurant,Bubble Tea Shop,Japanese Restaurant,Pizza Place,Fast Food Restaurant,Italian Restaurant
12,St. James Town,2,Coffee Shop,Café,American Restaurant,Gastropub,Cocktail Bar,Restaurant,Creperie,Cosmetics Shop,Clothing Store,Lingerie Store
13,Berczy Park,2,Coffee Shop,Cocktail Bar,Café,Cheese Shop,Bakery,Seafood Restaurant,Restaurant,Beer Bar,Lounge,Park
